In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/chatbot/CosineSimilarity/')

Mounted at /content/drive


In [2]:
# 형태소 분석기 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.
/content/drive/MyDrive/chatbot/CosineSimilarity/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 453 kB 56.7 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-07-31 14:47:00--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c0:3470, 2406:da00:ff00::22c5:2ef4, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?response-c

In [3]:
import pandas as pd
from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm


In [4]:
os.chdir('/content/drive/MyDrive/chatbot/CosineSimilarity/')
!ls

book.gsheet  book_jun.csv  Mecab-ko-for-Google-Colab  recommendation.ipynb


In [5]:
df = pd.read_csv('./book_jun.csv')
df = df.dropna()
df.head()

,Unnamed: 0,isbn,summary
0,0,2090000050714,김탁환 작가의 &lt;거짓말이다&gt;와 꽃다발을 함께 구입할 수 있는 기획 상품....
1,1,8809104721123,"독도가 명백한 우리 땅임을 밝혀주는 역사적 기록, 스토리맵 독도지도. 독도가 한국의..."
2,2,8809125901368,이영권 박사가 이야기 하는 ‘네트워크 마케팅’. 앞으로의 유통 방식은 네트워크 마케...
3,3,8809332972250,엄마의 다이어리 가계부. 씀의 컨셉은 세월을 기록하는 한 권의 책으로 가족의 10년...
4,4,8809332972649,냉장고에 걸어서 사용하는 캘린더형 가계부. 면에 일주일 단위로 적으며 돈의 흐름을 ...


In [6]:
mecab = Mecab()

tagged_corpus_list = []

for index, row in tqdm(df.iterrows(), total=len(df)):
  text = row['summary']
  tag = str(row['isbn'])

  tagged_corpus_list.append(TaggedDocument(tags=[tag], words=mecab.morphs(text)))

print('문서의 수 :', len(tagged_corpus_list))

100%|██████████| 40201/40201 [00:32<00:00, 1221.58it/s]

문서의 수 : 40201


In [7]:
tagged_corpus_list[100]

TaggedDocument(words=['&', 'lt', ';', '에픽', '#', '08', '&', 'gt', ';', '의', '제호', '는', '‘', '백', '년', '의', '고독', '’', '이', '다', '.', '작가', '마르케스', '가', '환상', '과', '실제', '의', '경계', '를', '넘나들', '며', '고립', '되', '고', '혼란', '스러운', '세계', '를', '고찰', '한', '것', '처럼', ',', '현실', '위', '에', '펼쳐진', '비', '현실', '적', '장면', '들', '을', '직시', '함', '으로써', '새로운', '삶', '의', '지도', '를', '그리', '는', '방법', '을', '우리', '에게', '보여준다', '.'], tags=['9772733807003'])

In [8]:
from gensim.models.doc2vec import Doc2Vec
import gensim

In [9]:
gensim.__version__

'3.6.0'

In [10]:
model = Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)
print(f"Tag Size: {len(model.docvecs.doctags.keys())}", end=' / ')

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

# 모델 저장
model.save('recommendation.doc2vec')

Tag Size: 40201 / 

In [11]:
#model = Doc2Vec.load('recommendation.doc2vec')

In [12]:
isbns = df['isbn']

results = []
for isbn in isbns:
    similar_books = model.docvecs.most_similar(isbn)
    result_books = []

    for similar_book in similar_books:
        result_books.append(similar_book[0])
    result_books = ",".join(result_books)
    results.append(result_books)
    
df["similar"] = results
df.head()

,Unnamed: 0,isbn,summary,similar
0,0,2090000050714,김탁환 작가의 &lt;거짓말이다&gt;와 꽃다발을 함께 구입할 수 있는 기획 상품....,"9791131988725,9791155053621,9791191785623,9791..."
1,1,8809104721123,"독도가 명백한 우리 땅임을 밝혀주는 역사적 기록, 스토리맵 독도지도. 독도가 한국의...","9791166031274,9788998241063,9788992449984,9791..."
2,2,8809125901368,이영권 박사가 이야기 하는 ‘네트워크 마케팅’. 앞으로의 유통 방식은 네트워크 마케...,"9788971967867,9788954238595,9788984331297,9791..."
3,3,8809332972250,엄마의 다이어리 가계부. 씀의 컨셉은 세월을 기록하는 한 권의 책으로 가족의 10년...,"9788954624077,9791127409692,9791197117312,9788..."
4,4,8809332972649,냉장고에 걸어서 사용하는 캘린더형 가계부. 면에 일주일 단위로 적으며 돈의 흐름을 ...,"8809482280434,8809482280120,8809482280007,9788..."


In [13]:
df.to_csv(path_or_buf='./similar_result.csv', index=False) 

In [14]:
df = pd.read_csv('./similar_result.csv')
df.head()

,Unnamed: 0,isbn,summary,similar
0,0,2090000050714,김탁환 작가의 &lt;거짓말이다&gt;와 꽃다발을 함께 구입할 수 있는 기획 상품....,"9791131988725,9791155053621,9791191785623,9791..."
1,1,8809104721123,"독도가 명백한 우리 땅임을 밝혀주는 역사적 기록, 스토리맵 독도지도. 독도가 한국의...","9791166031274,9788998241063,9788992449984,9791..."
2,2,8809125901368,이영권 박사가 이야기 하는 ‘네트워크 마케팅’. 앞으로의 유통 방식은 네트워크 마케...,"9788971967867,9788954238595,9788984331297,9791..."
3,3,8809332972250,엄마의 다이어리 가계부. 씀의 컨셉은 세월을 기록하는 한 권의 책으로 가족의 10년...,"9788954624077,9791127409692,9791197117312,9788..."
4,4,8809332972649,냉장고에 걸어서 사용하는 캘린더형 가계부. 면에 일주일 단위로 적으며 돈의 흐름을 ...,"8809482280434,8809482280120,8809482280007,9788..."
